In [7]:
import os

data_dir = r"C:\ComSL\mongolia_dataset"
clips_dir = os.path.join(data_dir, "clips")

tsv_files = [
    "dev.tsv",
    "invalidated.tsv",
    "other.tsv",
    "test.tsv",
    "train.tsv",
    "validated.tsv"
]

print("Số lượng dòng trong các file tsv:")
for tsv_file in tsv_files:
    tsv_path = os.path.join(data_dir, tsv_file)  # Sửa lại ở đây, không phải clips_dir
    try:
        with open(tsv_path, 'r', encoding='utf-8') as f:
            line_count = sum(1 for line in f)
        print(f"{tsv_file}: {line_count}")
    except Exception as e:
        print(f"{tsv_file}: Lỗi - {e}")

audio_extensions = ('.mp3', '.wav', '.flac', '.ogg')
audio_count = sum(1 for f in os.listdir(clips_dir) if f.lower().endswith(audio_extensions))
print(f"\nSố lượng audio trong folder clips: {audio_count}")

Số lượng dòng trong các file tsv:
dev.tsv: 1762
invalidated.tsv: 581
other.tsv: 1312
test.tsv: 1760
train.tsv: 2019
validated.tsv: 6372

Số lượng audio trong folder clips: 8262


In [11]:
tsv_files = [
    "covost_v2.mn_en.tsv"
]

print("Số lượng dòng trong các file tsv:")
for tsv_file in tsv_files:
    tsv_path = os.path.join(data_dir, tsv_file)  
    try:
        with open(tsv_path, 'r', encoding='utf-8') as f:
            line_count = sum(1 for line in f)
        print(f"{tsv_file}: {line_count}")
    except Exception as e:
        print(f"{tsv_file}: Lỗi - {e}")

Số lượng dòng trong các file tsv:
covost_v2.mn_en.tsv: 6372


In [8]:
import os
import csv

data_dir = r"C:\ComSL\mongolia_dataset"
clips_dir = os.path.join(data_dir, "clips")
validated_tsv = os.path.join(data_dir, "validated.tsv")

# Lấy danh sách file audio thực tế trong folder clips
audio_files = set(os.listdir(clips_dir))

# Đếm số lượng path thực sự tồn tại trong folder clips
count_exist = 0
total_rows = 0

with open(validated_tsv, 'r', encoding='utf-8') as tsvfile:
    reader = csv.DictReader(tsvfile, delimiter='\t')
    for row in reader:
        total_rows += 1
        audio_path = row['path']
        if audio_path in audio_files:
            count_exist += 1

print(f"Số lượng path có thật trong folder clips: {count_exist}/{total_rows}")

Số lượng path có thật trong folder clips: 6371/6371


In [9]:
import os
import csv

data_dir = r"C:\ComSL\mongolia_dataset"
clips_dir = os.path.join(data_dir, "clips")
validated_tsv = os.path.join(data_dir, "validated.tsv")

# Đọc tất cả path hợp lệ từ validated.tsv
valid_paths = set()
with open(validated_tsv, 'r', encoding='utf-8') as tsvfile:
    reader = csv.DictReader(tsvfile, delimiter='\t')
    for row in reader:
        valid_paths.add(row['path'])

# Duyệt tất cả file trong clips, xoá file không có trong valid_paths
deleted_count = 0
for file_name in os.listdir(clips_dir):
    if file_name not in valid_paths:
        file_path = os.path.join(clips_dir, file_name)
        try:
            os.remove(file_path)
            deleted_count += 1
        except Exception as e:
            print(f"Lỗi xoá {file_name}: {e}")

print(f"Đã xoá {deleted_count} file không có trong validated.tsv.")

Đã xoá 1891 file không có trong validated.tsv.


In [10]:
import os
import csv
import random
import shutil

data_dir = r"C:\ComSL\mongolia_dataset"
clips_dir = os.path.join(data_dir, "clips")
validated_tsv = os.path.join(data_dir, "validated.tsv")

# Output folders and TSV files
splits = {
    "train": os.path.join(data_dir, "clips_train"),
    "test": os.path.join(data_dir, "clips_test"),
    "dev": os.path.join(data_dir, "clips_dev"),
}
tsv_out = {
    "train": os.path.join(data_dir, "train.tsv"),
    "test": os.path.join(data_dir, "test.tsv"),
    "dev": os.path.join(data_dir, "dev.tsv"),
}

# Read validated.tsv
with open(validated_tsv, 'r', encoding='utf-8') as tsvfile:
    rows = list(csv.DictReader(tsvfile, delimiter='\t'))

# Shuffle and split
random.shuffle(rows)
n = len(rows)
n_train = int(n * 0.8)
n_test = int(n * 0.1)
n_dev = n - n_train - n_test

split_rows = {
    "train": rows[:n_train],
    "test": rows[n_train:n_train+n_test],
    "dev": rows[n_train+n_test:]
}

# Write TSV files and copy/move files
for split in ['train', 'test', 'dev']:
    # Ghi tsv
    with open(tsv_out[split], 'w', encoding='utf-8', newline='') as f_out:
        writer = csv.DictWriter(f_out, fieldnames=rows[0].keys(), delimiter='\t')
        writer.writeheader()
        writer.writerows(split_rows[split])
    print(f"Đã tạo file {tsv_out[split]} với {len(split_rows[split])} dòng.")

    # Tạo folder nếu chưa có & move audio
    os.makedirs(splits[split], exist_ok=True)
    for row in split_rows[split]:
        src = os.path.join(clips_dir, row['path'])
        dst = os.path.join(splits[split], row['path'])
        if os.path.exists(src):
            shutil.move(src, dst)  # hoặc shutil.copy nếu muốn copy thay vì move
        # Không cần else, vì có thể đã move trước đó

print("Hoàn tất tách audio và tạo các file tsv.")

Đã tạo file C:\ComSL\mongolia_dataset\train.tsv với 5096 dòng.
Đã tạo file C:\ComSL\mongolia_dataset\test.tsv với 637 dòng.
Đã tạo file C:\ComSL\mongolia_dataset\dev.tsv với 638 dòng.
Hoàn tất tách audio và tạo các file tsv.


In [ ]:
# # tạo thư mục giống paper yêu cầu 
# mkdir -p /mnt/c/ComSL/mongolia_dataset/covost/mn_en
# cp /mnt/c/ComSL/mongolia_dataset/covost_v2.mn_en.*.tsv /mnt/c/ComSL/mongolia_dataset/covost/mn_en/

# Download model

In [1]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())

2.7.0+cpu
False


In [3]:
!pip install protobuf==3.20.3
!pip install tiktoken


  Obtaining dependency information for protobuf==3.20.3 from https://files.pythonhosted.org/packages/8d/14/619e24a4c70df2901e1f4dbc50a6291eb63a759172558df326347dce1f0d/protobuf-3.20.3-py2.py3-none-any.whl.metadata
   ---------------------------------------- 0.0/162.1 kB ? eta -:--:--
   -- ------------------------------------- 10.2/162.1 kB ? eta -:--:--
   -------------------- ------------------- 81.9/162.1 kB 1.2 MB/s eta 0:00:01
   ---------------------------------------- 162.1/162.1 kB 1.4 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Obtaining dependency information for tiktoken from https://files.pythonhosted.org/packages/6f/07/c67ad1724b8e14e2b4c8cca04b15da158733ac60136879131db05dda7c30/tiktoken-0.9.0-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/893.9 kB ? eta -:--:--
    -------------------------------------- 20.5/893.9 kB 640.0 kB/s eta 0:00:02
   --- ----------------------------------- 71.7/893.9 kB 975.2 kB/s eta 0:00:01
   ------------ --------------------------- 276.5/893.9 kB 2.4 MB/s eta 0:00:01
   -------------------------- ------------- 583.7/893.9 kB 3.7 MB/s eta 0:00:01
   ---------------------------------------- 893.9/893.9 kB 5.1 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

model = MBartForConditionalGeneration.from_pretrained(
    "facebook/mbart-large-50-many-to-many-mmt",
    cache_dir="/mnt/c/ComSL/models/mbart-large-50-many-to-many-mmt"
)
tokenizer = MBart50TokenizerFast.from_pretrained(
    "facebook/mbart-large-50-many-to-many-mmt",
    cache_dir="/mnt/c/ComSL/models/mbart-large-50-many-to-many-mmt"
)


ValueError: Converting from SentencePiece and Tiktoken failed, if a converter for SentencePiece is available, provide a model path with a SentencePiece tokenizer.model file.Currently available slow->fast converters: ['AlbertTokenizer', 'BartTokenizer', 'BarthezTokenizer', 'BertTokenizer', 'BigBirdTokenizer', 'BlenderbotTokenizer', 'CamembertTokenizer', 'CLIPTokenizer', 'CodeGenTokenizer', 'ConvBertTokenizer', 'DebertaTokenizer', 'DebertaV2Tokenizer', 'DistilBertTokenizer', 'DPRReaderTokenizer', 'DPRQuestionEncoderTokenizer', 'DPRContextEncoderTokenizer', 'ElectraTokenizer', 'FNetTokenizer', 'FunnelTokenizer', 'GPT2Tokenizer', 'HerbertTokenizer', 'LayoutLMTokenizer', 'LayoutLMv2Tokenizer', 'LayoutLMv3Tokenizer', 'LayoutXLMTokenizer', 'LongformerTokenizer', 'LEDTokenizer', 'LxmertTokenizer', 'MarkupLMTokenizer', 'MBartTokenizer', 'MBart50Tokenizer', 'MPNetTokenizer', 'MobileBertTokenizer', 'MvpTokenizer', 'NllbTokenizer', 'OpenAIGPTTokenizer', 'PegasusTokenizer', 'Qwen2Tokenizer', 'Realm

In [ ]:
'''
1. sửa lỗi hàm on_validation_epoch_end không nhận tham số outputs nữa.
2. 
'''